In [403]:
# Import dependencies
import pandas as pd
import numpy as np

# Load in csv files
school_data_to_load="Resources/schools_complete.csv"
student_data_to_load="Resources/students_complete.csv"

# Convert csv files into pandas data frames
school_data_df=pd.read_csv(school_data_to_load)
student_data_df=pd.read_csv(student_data_to_load)

In [404]:
# Join the school data and student data tables on the 'school_name' column
full_data_df=pd.merge(student_data_df, school_data_df, how="left", on="school_name")

In [405]:
# Rename column titles and rearrange column order to be more natural
full_data_df=full_data_df.rename(columns={"student_name": "Student Name", "gender": "Gender", "grade": "Grade", "school_name": "School Name", "reading_score": "Reading Score", "math_score": "Math Score", "type": "School Type", "size": "School Size", "budget": "School Budget"})
full_data_df=full_data_df[["Student ID", "Student Name", "Gender", "Grade", "Reading Score", "Math Score", "School ID", "School Name", "School Type", "School Size", "School Budget"]]
full_data_df.head()

,Student ID,Student Name,Gender,Grade,Reading Score,Math Score,School ID,School Name,School Type,School Size,School Budget
0,0,Paul Bradley,M,9th,66,79,0,Huang High School,District,2917,1910635
1,1,Victor Smith,M,12th,94,61,0,Huang High School,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,90,60,0,Huang High School,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,67,58,0,Huang High School,District,2917,1910635
4,4,Bonnie Ray,F,9th,97,84,0,Huang High School,District,2917,1910635


In [406]:
# # Testing that columns don't have anything weird in them I can't see from the head() above. 
# # As long as they don't have anything weird going on, then I can be sure my summary statistcs
# # found below are reliable values. This entire section will be commented out at the end.

# # Test that Student ID column has no duplicates, no NaN values, and only integer values
# full_data_df['Student ID'].duplicated().unique() # Returns 'array([False])' which means no duplicate values in Student ID column
# full_data_df['Student ID'].isna().unique() # Returns 'array([False])' which means no NaN values in Student ID column
# full_data_df['Student ID'].dtypes # Returns 'int64' meaning each Student ID element is an integer.

# # Test that School ID has no NaN values and only integer values
# full_data_df['School ID'].unique() # Returns an array from 0 to 14
# full_data_df['School ID'].dtypes # Returns 'int64' meaning each School ID element is an integer

# # Test that for each unique School ID, there is a unique School Budget value associated with it
# full_data_df.groupby(['School ID', 'School Budget']).size() # Demonstrates there is a unique school budget value for each school ID, and also prints how many rows there are for each school

# # Test that the Math Score column has no NaN values and only integer values
# full_data_df['Math Score'].isna().unique() # Returns 'array([False])' which means no NaN values in Math Score column
# full_data_df['Math Score'].dtypes # Returns 'int64' meaning each Math Score is an integer

# # Test that the Reading Score column has no NaN values and only integer values
# full_data_df['Reading Score'].isna().unique() # Returns 'array([False])' which means no NaN values in Reading Score column
# full_data_df['Reading Score'].dtypes # Returns 'int64' meaning each Reading Score is an integer

In [407]:
# Calculate values of number of students passing Math and Reading, respectively

# Define counter variables, set initial values to 0
numPassMath = 0
numPassReading = 0

# Define simple for loops to count # of students passing in each subject respectively
for x in full_data_df['Math Score']:
    if x >= 70:
        numPassMath += 1

for x in full_data_df['Reading Score']:
    if x >= 70:
        numPassReading += 1

In [408]:
# Store variables for each metric I want to put in my district summary dataframe
totalSchools=full_data_df.nunique().loc['School ID']
totalStudents=full_data_df.nunique().loc['Student ID']
totalBudget=full_data_df['School Budget'].unique().sum()
meanMathScore=full_data_df['Math Score'].mean()
meanReadingScore=full_data_df['Reading Score'].mean()
percentPassMath=numPassMath/totalStudents
percentPassReading=numPassReading/totalStudents
overallPassRate=(percentPassMath + percentPassReading)/2 # NOTE: This yields a different value than what the homework specifies, but I'm pretty sure this formula is correct. Not sure why there is a discrepancy.

In [409]:
# Print district summary dataframe
district_summary_df=pd.DataFrame(
    {
         'Total Schools': totalSchools,
         'Total Students': '{:,}'.format(totalStudents),
         'Total Budget': '${:,.2f}'.format(totalBudget),
         'Average Math Score': meanMathScore,
         'Average Reading Score': meanReadingScore,
         '% Passing Math': '{0:.6f}'.format(percentPassMath*100),
         '% Passing Reading': '{0:.6f}'.format(percentPassReading*100),
         '% Overall Passing Rate': '{0:.6f}'.format(overallPassRate*100)
    }, 
    
    index=['Stats'])

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Stats,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.393158


In [410]:
# Store lists for each column I want to put into my school summary dataframe. Split up over 2 cells in Jupyter Notebook.

# Define relevant groupby object to pull list information from for both school type, school name, and number of students
name_type=full_data_df.groupby(['School Name', 'School Type']).count() # Define relevant groupby object 
name_type_df=pd.DataFrame(name_type) # Define data frame for groupby object so I can pull the columns I want

# Pull my school names, school types, and number of students data from this dataframe
schoolNames=name_type_df.index.get_level_values('School Name').tolist() # Pull first index into a list for school names
schoolTypes=name_type_df.index.get_level_values('School Type').tolist() # Pull second index into a list for school types
numStudents=name_type_df['Student ID'].tolist() # Pull student ID column numbers into a list for number of students

# Find total school budget using a different groupby object transferred to data frame. Note that school names will be alphabeitcal in both groupby objects so this method is sound.
name_budget=full_data_df.groupby(['School Name', 'School Budget']).count() # Call this group2 so I don't mess with my definition of group above
name_budget_df=pd.DataFrame(name_budget)
totalSchoolBudget=name_budget_df.index.get_level_values('School Budget').tolist()

# Calculate per student budget
perStudentBudget=[] # Define list of student budgets
tracker = 0 # Define tracker variable

for j in range(len(totalSchoolBudget)):
    tracker = totalSchoolBudget[j]/numStudents[j]
    perStudentBudget.append(tracker)

# Calculate average math and reading scores per school
averages=full_data_df.groupby(['School Name']).mean()
averages_df=pd.DataFrame(averages)
meanReadingScorePerSchool=averages_df['Reading Score'].tolist()
meanMathScorePerSchool=averages_df['Math Score'].tolist()

In [411]:
# Calculate perecent passing math and reading by school with these steps.
    # Step 1: Create data frames which just have students passing in Math and Reading, respectively
    # Step 2: Play with data frame to get number of students passing in Math and Reading, respectively, for each school.
    # Step 3: Take these values and divide by number of students at school (already calculated above).
    
# Step 1: Create data frames which just have students passing in Math and Reading, respectively
passMath=full_data_df['Math Score'] >= 70 # Define Boolean variable for what it means to be passing math
passReading=full_data_df['Reading Score'] >= 70 # Define Boolean variable for what it means to be passing reading

passing_math_df=full_data_df[passMath]
passing_reading_df=full_data_df[passReading]

# Step 2: Play with data frame to get number of students passing in Math and Reading, respectively, for each school.
# Do this for Math
name_math=passing_math_df.groupby(['School Name', 'Math Score']).count()
name_math_df=pd.DataFrame(name_math)
name_math_df=name_math_df.rename(columns={'Student ID': 'Number of Students'})
name_math_df=pd.DataFrame(name_math_df['Number of Students']) # Reduces complexity of data frame in terms of number of columns

name_math_df=name_math_df.reset_index(level='Math Score') # Reduces indexing complexity
name_math_df=name_math_df.reset_index(level='School Name') # Reducex indexing complexity

sum_math=name_math_df.groupby(['School Name']).sum() # Define a second group by object to get sum data
sum_math_df=pd.DataFrame(sum_math)
numPassMathPerSchool=sum_math_df['Number of Students'].tolist()

# Do this for Reading
name_read=passing_reading_df.groupby(['School Name', 'Reading Score']).count()
name_read_df=pd.DataFrame(name_read)
name_read_df=name_read_df.rename(columns={'Student ID': 'Number of Students'})
name_read_df=pd.DataFrame(name_read_df['Number of Students']) # Reduces complexity of data frame in terms of number of columns

name_read_df=name_read_df.reset_index(level='Reading Score') # Reduces indexing complexity
name_read_df=name_read_df.reset_index(level='School Name') # Reducex indexing complexity

sum_read=name_read_df.groupby(['School Name']).sum() # Define a second group by object to get sum data
sum_read_df=pd.DataFrame(sum_read)
numPassReadingPerSchool=sum_read_df['Number of Students'].tolist()

# Step 3: Take these values and divide by number of students at school (already calculated above).
percentPassMathPerSchool=[] # Define list of percentage of students passing math
percentPassReadingPerSchool=[] # Define list of percentage of students passing reading
overallPassRatePerSchool=[] # Define list of overall passing rates 
mathTracker=0 # Define math tracker variable
readingTracker=0 # Define reading tracker variable
overallTracker=0

for j in range(len(schoolNames)):
    mathTracker=numPassMathPerSchool[j]/numStudents[j]
    readingTracker=numPassReadingPerSchool[j]/numStudents[j]
    overallTracker=(mathTracker+readingTracker)/2
    percentPassMathPerSchool.append(mathTracker)
    percentPassReadingPerSchool.append(readingTracker)    
    overallPassRatePerSchool.append(overallTracker)

In [412]:
# Prep formatting of values in summary data frame
numStudents=['{:,}'.format(x) for x in numStudents]
totalSchoolBudget=['${:,.2f}'.format(x) for x in totalSchoolBudget]
perStudentBudget=['${:,.2f}'.format(x) for x in perStudentBudget]
percentPassMathPerSchool=['{0:.6f}'.format(100*x) for x in percentPassMathPerSchool]
percentPassReadingPerSchool=['{0:.6f}'.format(100*x) for x in percentPassReadingPerSchool]
overallPassRatePerSchool=['{0:.6f}'.format(100*x) for x in overallPassRatePerSchool]

In [413]:
# Print school summary dataframe
school_summary_df=pd.DataFrame(
    {
        'School Name': schoolNames,
        'School Type': schoolTypes,
        'Number of Students': numStudents,
        'Total Budget': totalSchoolBudget,
        'Budget Per Student': perStudentBudget,
        'Average Math Score': meanMathScorePerSchool,
        'Average Reading Score': meanReadingScorePerSchool,
        '% Passing Math': percentPassMathPerSchool,
        '% Passing Reading': percentPassReadingPerSchool,
        '% Overall Passing Rate': overallPassRatePerSchool
    })

school_summary_df

# 'Total Schools': totalSchools,
#          'Total Students': '{:,}'.format(totalStudents),
#          'Total Budget': '${:,.2f}'.format(totalBudget),
#          'Average Math Score': meanMathScore,
#          'Average Reading Score': meanReadingScore,
#          '% Passing Math': '{0:.6f}'.format(percentPassMath*100),
#          '% Passing Reading': '{0:.6f}'.format(percentPassReading*100),
#          '% Overall Passing Rate': '{0:.6f}'.format(overallPassRate*100)

,School Name,School Type,Number of Students,Total Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
1,Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
2,Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
3,Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308
4,Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
5,Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,73.807983
6,Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,94.379391
7,Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
8,Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
